# Práctica - Segundo parcial

Alejandro Noel Hernández Gutiérrez

Ingeniería de características

Docente Carlos Augusto Arellano Muro

Instituto tecnológico y de estudios superiores de occidente

En el siguiete archivo practica2.xlsx, se muestran evaluaciones de una biblioteca en España, el objetivo es encontrar una relación entre las evaluaciones específicas (Estado del material, Organización, Trato, etc.) y la Valoración Global.
1. Elimine las observaciones que no tengan un dato en VALORACION GLOBAL
2. Codifique la variable '<5 CAUSA'
3. Evalúe las variables con una transformación de potencia para compensar el sesgo lo mejor posible
4. Impute los valores faltantes por media, mediana, valores aleatorios o moda, según convenga
5. Encuentre un modelo que relacione las evaluaciones específicas junto con la variable codificada de '<5 CAUSA' con el target VALORACION GLOBAL (El modelo recomendado es Árboles de decisión sklearn.tree.DecisionTreeRegresor)

In [70]:
# Librerías
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor

In [74]:
# Lectura de archivo
excel = pd.read_excel("practica2.xlsx")
excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ADECUACIÓN COLECCIÓN  985 non-null    object
 1   ESTADO MATERIAL       1019 non-null   object
 2   ORGANIZACIÓN          1010 non-null   object
 3   ORDENACIÓN            1011 non-null   object
 4   TRATO                 1079 non-null   object
 5   INFORMACIÓN.1         937 non-null    object
 6   HORARIO               1030 non-null   object
 7   WIFI.1                578 non-null    object
 8   ESTADO                1073 non-null   object
 9   LIMPIEZA              1084 non-null   object
 10  <5 CAUSA              203 non-null    object
 11  VALORACION GLOBAL     1097 non-null   object
dtypes: object(12)
memory usage: 104.4+ KB


In [75]:
# Elimine las observaciones que no tengan un dato en VALORACION GLOBAL
excel = excel.dropna(subset=['VALORACION GLOBAL'])
excel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097 entries, 0 to 1111
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ADECUACIÓN COLECCIÓN  979 non-null    object
 1   ESTADO MATERIAL       1013 non-null   object
 2   ORGANIZACIÓN          1004 non-null   object
 3   ORDENACIÓN            1005 non-null   object
 4   TRATO                 1072 non-null   object
 5   INFORMACIÓN.1         933 non-null    object
 6   HORARIO               1025 non-null   object
 7   WIFI.1                575 non-null    object
 8   ESTADO                1069 non-null   object
 9   LIMPIEZA              1079 non-null   object
 10  <5 CAUSA              203 non-null    object
 11  VALORACION GLOBAL     1097 non-null   object
dtypes: object(12)
memory usage: 111.4+ KB


In [76]:
# Observamos que ya no existen valores null para la columna valoración global
excel['VALORACION GLOBAL'].isnull().values.any()

False

In [77]:
excel.describe()

,ADECUACIÓN COLECCIÓN,ESTADO MATERIAL,ORGANIZACIÓN,ORDENACIÓN,TRATO,INFORMACIÓN.1,HORARIO,WIFI.1,ESTADO,LIMPIEZA,<5 CAUSA,VALORACION GLOBAL
count,979,1013,1004,1005,1072,933,1025,575,1069,1079,203,1097
unique,13,13,13,13,14,13,12,13,13,12,201,13
top,8,8,10,10,10,10,10,No uso,10,10,10,9
freq,248,247,285,312,637,315,303,97,314,457,3,370


In [78]:
# codificar la variable '<5 CAUSA ' usando Feature hashing
vectorizer = HashingVectorizer(n_features=100)
text_matrix = vectorizer.fit_transform(excel['<5 CAUSA ']).toarray()

KeyError: '<5 CAUSA '

# Codifique la variable '<5 CAUSA'  
Codificación por el valor que escogiste
Podría ser Top words, kmeans

In [21]:
# Sustituir nan values for option other
excel['<5 CAUSA'].isna().sum()
# 893 de 1097 valores son NA

894

In [26]:
# Rellenamos los NaN values con causa desconocida o Otra
excel['<5 CAUSA'] = excel['<5 CAUSA'].fillna('Otra')

In [27]:
# No más NA:
excel['<5 CAUSA'].isna().sum()

0

In [38]:
# En la exploración se encuentran valores extraños, posiblemente equivocación de los usuarios: [7, 9 , 10, Na, no, U], reemplazamos por 'Sin información'
excel['<5 CAUSA'].isin([7, 9 , 10,'Na' , 'no', 'U']).sum()

8

In [40]:
# [7, 9 , 10, Na, no, U], reemplazamos por 'Sin información'
excel=excel.mask(excel['<5 CAUSA'].isin([7, 9 , 10,'Na' , 'no', 'U']), 'Sin información')

In [42]:
# Contar para asegurar que esos valores ya son 0
excel['<5 CAUSA'].isin([7, 9 , 10,'Na' , 'no', 'U']).sum()

0

In [46]:
# crear matriz BoW
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(excel['<5 CAUSA'])
bow_matrix

<1097x1016 sparse matrix of type '<class 'numpy.int64'>'
	with 3839 stored elements in Compressed Sparse Row format>

In [60]:
# Calcular las frecuencias de las palabras en el corpus
frecuencias = np.array(np.sum(bow_matrix, axis=0))[0]
word_count = pd.DataFrame({'word':vectorizer.get_feature_names_out(), 'count':frecuencias})
word_count_sorted = word_count.sort_values('count', ascending=False)
word_count_sorted

,word,count
655,otra,897
234,de,184
497,la,154
316,en,123
622,no,108
...,...,...
396,finales,1
397,fines,1
398,fluorescentes,1
399,fondo,1


## Después de visualizar el dataframe de las palabras más repetidas, se escogen de entre ellas sustantivos o adjetivos que puedan indicar causa de inconformidad
# Manualmente se estiman unos 6 grupos de categorías (un enter por cada grupo)

### Alta temperatura, o temperatura en general:
- calor
- calefacción
- acondicionado
- temperatura
- climatización

### Baja temperatura:
- frío
- refrigeración

### Iluminación o luz:
- luz
- iluminación

### Ventilación o covid
- covid
- ventanas
- ventilación

### Fallas en el servicio
- wifi
- servicio
- préstamo
- tiempo
- tarde
- espacio
- personal

### Equipamiento
- enchufes
- intercentros
- ordenadores
- ascensor

### Todo lo demás se irá a 0
- información
- otros



In [62]:
# Ordenar las palabras por frecuencia y seleccionar las palabras más repetidas como nuevas categorías
new_categories = ['calor','calefacción','acondicionado','temperatura','climatización','frío','refrigeración','luz','iluminación','covid','ventanas','ventilación','wifi','servicio','préstamo','tiempo','tarde','espacio','personal','enchufes','intercentros','ordenadores','ascensor']
new_categories

['calor',
 'calefacción',
 'acondicionado',
 'temperatura',
 'climatización',
 'frío',
 'refrigeración',
 'luz',
 'iluminación',
 'covid',
 'ventanas',
 'ventilación',
 'wifi',
 'servicio',
 'préstamo',
 'tiempo',
 'tarde',
 'espacio',
 'personal',
 'enchufes',
 'intercentros',
 'ordenadores',
 'ascensor']

In [64]:
# Crear una matriz de características binarias indicando la presencia o ausencia de las palabras más repetidas
# La dimensión 
new_feature_matrix = np.zeros((len(excel), new_categories.__len__()))
# cada array tiene 23 ceros
new_feature_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [69]:
# Crear una matriz de características binarias indicando la presencia o ausencia de las palabras más repetidas
for i, doc in enumerate(excel['<5 CAUSA']):
    words = doc.split()
    for j, category in enumerate(new_categories):
        # cada array tiene 23 ceros, se pondrá un 1 cada que category (de la lista new_categories) esté dentro de las palabras
        if category in words:
            new_feature_matrix[i][j] = 1
new_feature_matrix[:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
# Concatenar la nueva matriz de características con la matriz BoW original
new_bow_matrix = np.concatenate((bow_matrix.toarray(), new_feature_matrix), axis=1)